<a href="https://colab.research.google.com/github/bendavidsteel/trade-democratization/blob/master/sensitivity_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch==1.5.1+cu101 torchvision==0.6.1+cu101 -f https://download.pytorch.org/whl/torch_stable.html
!pip install torch-scatter==2.0.4+cu101 -f https://pytorch-geometric.com/whl/torch-1.5.0.html
!pip install torch-sparse==0.6.5+cu101 -f https://pytorch-geometric.com/whl/torch-1.5.0.html
!pip install torch-cluster==1.5.5+cu101 -f https://pytorch-geometric.com/whl/torch-1.5.0.html
!pip install torch-spline-conv==1.2.0+cu101 -f https://pytorch-geometric.com/whl/torch-1.5.0.html
!pip install torch-geometric

In [ ]:
import itertools
import math
import os
import random

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch_geometric as geo
import tqdm

from google.colab import drive
drive.mount('/content/drive')

In this notebook I'd like to start doing some sensitivity analysis of the model, i.e. seeing if it reacts in the way one would expect to changes in bilateral relationships.

In [ ]:
class RecurGraphNet(torch.nn.Module):
    def __init__(self, num_node_features, num_edge_features, num_output_features):
        super().__init__()

        conv_layer_size = 32
        lstm_layer_size = 32

        # graph convolutional layer to create graph representation
        conv_lin = torch.nn.Linear(num_edge_features, num_node_features * conv_layer_size)
        self.conv = geo.nn.NNConv(num_node_features, conv_layer_size, conv_lin)

        # lstm to learn sequential patterns
        self.lstm = torch.nn.LSTM(conv_layer_size, lstm_layer_size, dropout=0.5)

        # initial trainable hidden state for lstm
        self.lstm_h_s = torch.nn.Linear(num_output_features, lstm_layer_size)
        self.lstm_c_s = torch.nn.Linear(num_output_features, lstm_layer_size)

        # final linear layer to allow full expressivity for regression after tanh activation in lstm
        self.final_linear = torch.nn.Linear(lstm_layer_size, num_output_features)

    def forward(self, input):
        initial, sequence = input.initial, input.sequence
        
        # create graph representation
        graph_collection = []
        for idx in range(len(sequence)):
            x, edge_index, edge_attr = sequence[idx].x, sequence[idx].edge_index, sequence[idx].edge_attr
            graph_step = torch.nn.functional.relu(self.conv(x, edge_index, edge_attr))
            graph_collection.append(graph_step)
        # provide graph representations as sequence to lstm
        graph_series = torch.stack(graph_collection)

        # recurrent stage
        # initial state of lstm is representation of target prior to this sequence
        lstm_output, _ = self.lstm(graph_series, (self.lstm_h_s(initial).unsqueeze(0), self.lstm_c_s(initial).unsqueeze(0)))

        # final activation is relu as this is for regression and the metrics of this dataset are all positive
        return self.final_linear(lstm_output)

In [ ]:
class Sequence():
    def __init__(self, initial, sequence, missing_mask, target):
        self.initial = initial
        self.sequence = sequence
        self.missing_mask = missing_mask
        self.target = target

    def to(self, device):
        self.initial = self.initial.to(device)
        self.missing_mask = self.missing_mask.to(device)
        self.target = self.target.to(device)
        for idx in range(len(self.sequence)):
            self.sequence[idx] = self.sequence[idx].to(device)
        return self

In [ ]:
@torch.no_grad()
def get_predictions(recur_seq, non_recur_seq):
    # get prediction of recurrent model
    recur_seq = recur_seq.to(device)
    recur_prediction = recur_model(recur_seq)

    # first data object in this sequence has prior demo conditions
    year_preds = []
    for idx in range(len(non_recur_seq)):
        data = non_recur_seq[idx].to(device)
        pred = non_recur_model(data)
        year_preds.append(pred)

        if idx + 1 < len(non_recur_seq):
            non_recur_seq[idx + 1].x[:, 2] = pred.squeeze(-1)

    non_recur_prediction = torch.stack(year_preds)

    return recur_prediction, non_recur_prediction

Lets create a simple scenario with three countries, with distinct democratization differences, and see how changes in their relationships affects their democratic tendencies over time. Lets call them A-Io, Thu and Anarres.

In [ ]:
# Anarres
anarres_initial = 0.95
# A-Io
aio_initial = 0.55
# Thu
thu_initial = 0.25